In [1]:
import os
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
import os
import requests
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup
import zipfile
import os
import glob
import boto3
from botocore.config import Config
import time
os.getcwd()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


'/home/ec2-user/SageMaker'

### Cloud Environment set-up

In [2]:
import sagemaker
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role() 
print(role)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


Couldn't call 'get_role' to get Role ARN from role name SageMaker-Climadata to get Role path.


arn:aws:iam::669361146924:role/SageMaker-Climadata


In [3]:
import requests
# Get the public IP address
public_ip_address = requests.get('https://api.ipify.org').text

print("public ip:", public_ip_address)

# Create a boto3 client
ec2 = boto3.client('ec2')

# The id of the security group to modify
security_group_id = ''

# Add the public IP address to the security group
ec2.authorize_security_group_ingress(
    GroupId=security_group_id,
    IpProtocol='tcp',
    FromPort=3306,  # Change this to your database's port
    ToPort=3306,  # Change this to your database's port
    CidrIp=f'{public_ip_address}/32'
)

public ip: 18.184.155.204


ClientError: An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 18.184.155.204/32, TCP, from port: 3306, to port: 3306, ALLOW" already exists

In [3]:
# getting metadata from s3
import boto3

# Set up AWS credentials and region
aws_access_key_id = ''
aws_secret_access_key = ''
region_name = 'eu-central-1'

session = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name
)
s3 = session.client('s3')
s3.download_file('climabucket.test', 'Metadata-DWD.csv', 'Metadata-DWD.csv')

In [6]:
# Load data into a pandas DataFrame
os.chdir('/home/ec2-user/SageMaker/')

metadata = pd.read_csv('Metadata-DWD.csv', sep=',', header=0)


### Access DWD data 

In [2]:
#getting the names of all the subfolders

base_url = "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/"

# Get the base page
response = requests.get(base_url)
directories = []

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a')
    for link in links:
        href = link.get('href')
        # Check if the link is a directory and not a parent directory link
        if href.endswith('/') and not href.startswith('../'):
            directories.append(href + "recent/")
else:
    print(f"Failed to get page from {base_url}. Status code: {response.status_code}")

# replace solar/recent with solar
directories[directories.index('solar/recent/')] = 'solar/'
print(directories)

['air_temperature/recent/', 'cloud_type/recent/', 'cloudiness/recent/', 'dew_point/recent/', 'extreme_wind/recent/', 'moisture/recent/', 'precipitation/recent/', 'pressure/recent/', 'soil_temperature/recent/', 'solar/', 'sun/recent/', 'visibility/recent/', 'weather_phenomena/recent/', 'wind/recent/', 'wind_synop/recent/']


In [7]:
# function to get a list of desired files from a directory, metadata df is required
# IMPORTANT: this relies on naming convention in DWD that id is 5 digits starting from 15th from
def get_files(url, metadata):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        all_names = [link.get('href') for link in soup.find_all('a')]
        # Get unique station ids as strings, padded with zeros
        station_ids = [str(id).zfill(5) for id in metadata['Stations_id'].unique()]
        # Output only those files that contain station id from metadata 
        desired_names = []

        for name in all_names:
            if '_akt' in name:
                substring = name[name.index('_akt')-5:name.index('_akt')]
                for id in station_ids:
                    if substring == id:
                        desired_names.append(name)    
        return desired_names
    else:
        print(f"Failed to get file list from {url}. Status code: {response.status_code}")
        return []

   
def download_and_extract_file(url, file):
    file_url = url + file
    response = requests.get(file_url)
    if response.status_code == 200:
        with open(file, 'wb') as f:
            f.write(response.content)
        with zipfile.ZipFile(file, 'r') as zip_ref:
            zip_ref.extractall()


def collapse_data_from_folder(folder_name):
    dir_df = pd.DataFrame()
   
    for file in os.listdir():
        if file.startswith('produkt') and file.endswith('txt'):
            incriment = pd.read_csv(file, sep=';', encoding='ISO-8859-1')
            dir_df = pd.concat([dir_df, incriment])
        else: pass

    return dir_df


def proc_measurements(df):
    #df['MESS_DATUM'] = pd.to_datetime(df['MESS_DATUM'], format='%Y%m%d%H')
    #df.set_index('MESS_DATUM', inplace=True)
    if 'eor' in df.columns:
        df.drop(columns = ['eor'], inplace = True)
    # drop values without quality flag
    if 'QB' in df.columns:
        df = df[df['QB'] != -999]
        
    # VERY IMPORTANT: SAMPLING OF WEATHER STATIONS OCCURS HERE 
    df = df[df['STATIONS_ID'].isin(metadata['Stations_id'])]
    return df

    
#function tot turn a contents of whole folder into a dataframe
#base_url is root if interesting web-data
#base_homedir is the directory where you want to store all the sub-folders data
#directory is the folder in dwd folder structure
#metadata is a df with metadata
def process_directory(base_url, base_homedir, directory, metadata):
    # free directory adrees from /recent/ part
    dir_i = directory.split('/')[0]
    url = base_url + directory
    files = get_files(url,metadata)
    # creating a folder to store all the files from desired directory
    os.chdir(base_homedir)
    os.mkdir(dir_i)
    os.chdir(dir_i)
    for file in files:
        download_and_extract_file(url, file)
    # deleting all the metadaten. mind that they contain only the info on number of missing values 
# but you can calculate them yourself. -999 is a missing value
    files = glob.glob('Metadaten*')
    for file in files:
        os.remove(file)
    dir_df = collapse_data_from_folder(folder_name = dir_i)    
    return dir_df #proc_measurements(dir_df)


In [10]:
base_url = "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/"

# Define the new directory to host all other folders
path = "/home/ec2-user/SageMaker/basis_23Feb"

# Create the directory
#os.mkdir(path)
project_wd = path

my_timer = []
my_timer.append(time.time())
air_temp = process_directory(base_url, project_wd, directories[0], metadata)
weather_phenomena = process_directory(base_url, project_wd, directories[12], metadata)
my_timer.append(time.time())
cloudiness = process_directory(base_url, project_wd, directories[2], metadata)
cloud_type = process_directory(base_url, project_wd, directories[1], metadata)
soil_temp = process_directory(base_url, project_wd, directories[8], metadata)
my_timer.append(time.time())
pressure = process_directory(base_url, project_wd, directories[7], metadata)
precipitation = process_directory(base_url, project_wd, directories[6], metadata)
my_timer.append(time.time())
sun = process_directory(base_url, project_wd, directories[10], metadata)
moisture = process_directory(base_url, project_wd, directories[5], metadata)
my_timer.append(time.time())
extreme_wind = process_directory(base_url, project_wd, directories[4], metadata)
wind = process_directory(base_url, project_wd, directories[13], metadata)
my_timer.append(time.time())

# Assuming my_timer is your list
differences = [t - my_timer[0] for t in my_timer]

# Print the results
for diff in differences:
    print(diff)


0.0
0.8285448551177979
2.163947343826294
3.5218117237091064
4.512378931045532
5.5096471309661865


In [4]:
#before opti
'''
0.0
2.9098830223083496
4.398086786270142
9.135090827941895
11.377177953720093
13.093568325042725
'''
#after opti
'''
0.0
0.8888800144195557
2.3557307720184326
3.2843453884124756
4.091327667236328
4.738299608230591
'''

4.738299608230591

In [20]:



'''
0 - ['air_temperature/recent/', 
1 -  'cloud_type/recent/', 
2 -  'cloudiness/recent/', 
3 -  'dew_point/recent/', 
4 -  'extreme_wind/recent/', 
5 -  'moisture/recent/', 
6 -  'precipitation/recent/', 
7 - 'pressure/recent/', 
8 - 'soil_temperature/recent/', 
9 - 'solar/', 
10- 'sun/recent/', 
11 - 'visibility/recent/', 
12 - 'weather_phenomena/recent/', 
13 - 'wind/recent/', 
14 - 'wind_synop/recent/']
'''
print(metadata.columns)
metadata['Stations_id'].unique()


Index(['Stations_id', 'von_datum', 'bis_datum', 'Stationshoehe', 'geoBreite',
       'geoLaenge', 'Stationsname', 'Bundeslandion_id', 'Unnamed: 8'],
      dtype='object')


array([ 399,  400,  403,  410,  420,  424,  427,  430,  433, 3987])

In [14]:
import boto3

# Create a Timestream write client
client = boto3.client('timestream-write')

# Specify the database name and the new table name
database_name = 'myclimab'
new_table_name = 'new_table'  # replace with your new table name

# Create the new table
response = client.create_table(
    DatabaseName=database_name,
    TableName=new_table_name,
    RetentionProperties={
        'MemoryStoreRetentionPeriodInHours': 24,
        'MagneticStoreRetentionPeriodInDays': 365
    }
)

print(response)

{'Table': {'Arn': 'arn:aws:timestream:eu-central-1:669361146924:database/myclimab/table/new_table', 'TableName': 'new_table', 'DatabaseName': 'myclimab', 'TableStatus': 'ACTIVE', 'RetentionProperties': {'MemoryStoreRetentionPeriodInHours': 24, 'MagneticStoreRetentionPeriodInDays': 365}, 'CreationTime': datetime.datetime(2024, 1, 16, 21, 59, 56, 954000, tzinfo=tzlocal()), 'LastUpdatedTime': datetime.datetime(2024, 1, 16, 21, 59, 56, 954000, tzinfo=tzlocal()), 'MagneticStoreWriteProperties': {'EnableMagneticStoreWrites': False}, 'Schema': {'CompositePartitionKey': [{'Type': 'MEASURE'}]}}, 'ResponseMetadata': {'RequestId': '5XRHNMNVAJLDYJWKA3EDV5LFPQ', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5XRHNMNVAJLDYJWKA3EDV5LFPQ', 'content-type': 'application/x-amz-json-1.0', 'content-length': '465', 'date': 'Tue, 16 Jan 2024 21:59:56 GMT'}, 'RetryAttempts': 0}}
